# Camera Manager

In [4]:
import carla
import numpy as np
import pygame
import weakref
import time

class CameraSetup:
    def __init__(self, vehicle, width=1280, height=720):
        self.sensor = None
        self.vehicle = vehicle
        self.width = width
        self.height = height
        self._surface = None
        
        # Initialize Pygame
        pygame.init()
        pygame.font.init()
        self.display = pygame.display.set_mode((width, height), pygame.HWSURFACE | pygame.DOUBLEBUF)
        pygame.display.set_caption("Camera View - Use arrows, WASD, and QE to adjust")
        
        # Camera parameters
        self.fov = 90.0
        self.height = 5.0
        self.pitch = -90.0
        
    def setup_camera(self, world):
        """Initial camera setup"""
        camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(self.width))
        camera_bp.set_attribute('image_size_y', str(self.height))
        camera_bp.set_attribute('fov', str(self.fov))
        
        # Initial transform
        camera_transform = carla.Transform(
            carla.Location(x=0.0, z=self.height),
            carla.Rotation(pitch=self.pitch)
        )
        
        # Spawn camera
        self.sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=self.vehicle)
        
        # Set up image callback
        weak_self = weakref.ref(self)
        self.sensor.listen(lambda image: self._process_image(weak_self, image))
        
    @staticmethod
    def _process_image(weak_self, image):
        """Convert image to pygame surface"""
        self = weak_self()
        if not self:
            return
        
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]  # Remove alpha channel
        array = array[:, :, ::-1]  # Convert from RGB to BGR
        self._surface = pygame.surfarray.make_surface(array.swapaxes(0, 1))

    def update_frame(self):
        """Update pygame display with current frame and parameters"""
        if self._surface is None:
            return
            
        self.display.blit(self._surface, (0, 0))
        
        # Display current parameters
        font = pygame.font.Font(None, 36)
        params = [
            f"FOV: {self.fov:.1f} (Left/Right)",
            f"Height: {self.height:.1f} (Up/Down)",
            f"Pitch: {self.pitch:.1f} (W/S)",
            "Press ESC to quit"
        ]
        
        y = 10
        for param in params:
            text = font.render(param, True, (255, 255, 255))
            self.display.blit(text, (10, y))
            y += 30
            
        pygame.display.flip()
        
    def update_camera_position(self):
        """Update camera transform based on current parameters"""
        transform = carla.Transform(
            carla.Location(x=0.0, z=self.height),
            carla.Rotation(pitch=self.pitch)
        )
        self.sensor.set_transform(transform)
        self.sensor.set_attribute('fov', str(self.fov))
        
    def destroy(self):
        """Cleanup"""
        if self.sensor:
            self.sensor.destroy()
        pygame.quit()

# Example usage:
"""
# 1. Setup
client = carla.Client("localhost", 2000)
world = client.get_world()
vehicles = world.get_actors().filter('vehicle.*')
vehicle = vehicles[0]

camera = CameraSetup(vehicle)
camera.setup_camera(world)

# 2. Main loop
running = True
while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                
    # Handle held keys
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        camera.fov = max(10, camera.fov - 1)
    if keys[pygame.K_RIGHT]:
        camera.fov = min(170, camera.fov + 1)
    if keys[pygame.K_UP]:
        camera.height += 0.1
    if keys[pygame.K_DOWN]:
        camera.height -= 0.1
    if keys[pygame.K_w]:
        camera.pitch = min(0, camera.pitch + 1)
    if keys[pygame.K_s]:
        camera.pitch = max(-180, camera.pitch - 1)
        
    # Update camera if any parameters changed
    camera.update_camera_position()
    
    # Update display
    camera.update_frame()
    time.sleep(0.01)  # Short sleep to prevent excessive CPU usage

# 3. Cleanup
camera.destroy()
"""

pygame 2.6.1 (SDL 2.28.4, Python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


'\n# 1. Setup\nclient = carla.Client("localhost", 2000)\nworld = client.get_world()\nvehicles = world.get_actors().filter(\'vehicle.*\')\nvehicle = vehicles[0]\n\ncamera = CameraSetup(vehicle)\ncamera.setup_camera(world)\n\n# 2. Main loop\nrunning = True\nwhile running:\n    # Handle events\n    for event in pygame.event.get():\n        if event.type == pygame.QUIT:\n            running = False\n        elif event.type == pygame.KEYDOWN:\n            if event.key == pygame.K_ESCAPE:\n                running = False\n                \n    # Handle held keys\n    keys = pygame.key.get_pressed()\n    if keys[pygame.K_LEFT]:\n        camera.fov = max(10, camera.fov - 1)\n    if keys[pygame.K_RIGHT]:\n        camera.fov = min(170, camera.fov + 1)\n    if keys[pygame.K_UP]:\n        camera.height += 0.1\n    if keys[pygame.K_DOWN]:\n        camera.height -= 0.1\n    if keys[pygame.K_w]:\n        camera.pitch = min(0, camera.pitch + 1)\n    if keys[pygame.K_s]:\n        camera.pitch = max(-18

In [5]:

# 1. Setup
client = carla.Client("localhost", 2000)
world = client.get_world()
vehicles = world.get_actors().filter('vehicle.*')
vehicle = vehicles[0]

camera = CameraSetup(vehicle)
camera.setup_camera(world)

# 2. Main loop
running = True
while running:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                
    # Handle held keys
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        camera.fov = max(10, camera.fov - 1)
    if keys[pygame.K_RIGHT]:
        camera.fov = min(170, camera.fov + 1)
    if keys[pygame.K_UP]:
        camera.height += 0.1
    if keys[pygame.K_DOWN]:
        camera.height -= 0.1
    if keys[pygame.K_w]:
        camera.pitch = min(0, camera.pitch + 1)
    if keys[pygame.K_s]:
        camera.pitch = max(-180, camera.pitch - 1)
        
    # Update camera if any parameters changed
    camera.update_camera_position()
    
    # Update display
    camera.update_frame()
    time.sleep(0.01)  # Short sleep to prevent excessive CPU usage

# 3. Cleanup
camera.destroy()


IndexError: vector::_M_range_check: __n (which is 0) >= this->size() (which is 0)

In [2]:
# Connect to the CARLA server
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

In [6]:
vehicles = world.get_actors().filter('vehicle.*')
if not vehicles:
    print("No vehicles found in the simulation. Please ensure a vehicle is spawned.")
vehicles 
vehicle = vehicles[0]  # Get the first vehicle
print(f"Found vehicle: {vehicle.type_id} at {vehicle.get_location()}")

# Initialize and setup camera
camera_manager = CameraManager(vehicle)
camera_manager.setup_camera(world)
print("Camera setup complete")   

Found vehicle: vehicle.tesla.model3 at Location(x=-327.453033, y=9.007962, z=0.224307)
Camera setup complete


In [3]:
running = True
while running:
    # Handle Pygame events for camera parameter adjustments
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            # Camera controls
            if event.key == pygame.K_UP:
                camera_manager.update_camera_params('height', 
                    camera_manager.camera_attributes['position'].location.z + 0.5)
            elif event.key == pygame.K_DOWN:
                camera_manager.update_camera_params('height', 
                    camera_manager.camera_attributes['position'].location.z - 0.5)
            elif event.key == pygame.K_LEFT:
                camera_manager.update_camera_params('fov', 
                    camera_manager.camera_attributes['fov'] - 5)
            elif event.key == pygame.K_RIGHT:
                camera_manager.update_camera_params('fov', 
                    camera_manager.camera_attributes['fov'] + 5)
            elif event.key == pygame.K_ESCAPE:
                running = False
    
    # Update camera display
    camera_manager.render()
    
    # Small delay to prevent excessive CPU usage
    time.sleep(0.01)

NameError: name 'pygame' is not defined

In [6]:
vehicle

NameError: name 'vehicle' is not defined

: 